# **Tarea 2**

## _Punto 1:_

### 1. Modelado del problema como un MDP

Modele este problema como un MDP. Detalle todos los elementos del MDP:

- **Estados**  
- **Recompensas**  
- **Acciones**  
- **Dinámica de transición**:  
  $$ p(s', r \mid s, a) \quad \forall s, s', r, a $$  
- **Factor de descuento**:  
  $$ \gamma $$

In [3]:
import numpy as np

Se definen los elementos del MDP, estableciendo 100 estados que representan las casillas numeradas del tablero. Cada estado corresponde a una casilla específica, desde la 1 hasta la 100. Además, se definen dos acciones posibles: avanzar (+1) o retroceder (−1), lo que significa que el jugador puede decidir moverse hacia adelante o hacia atrás. Esto permite parametrizar el sentido del movimiento, ya que permite al jugador evitar caer en serpientes o posicionarse mejor para subir por una escalera.

In [4]:
# Definir el número de estados y acciones
numEstados = 100  # Casillas del tablero
acciones = [-1, 1]  # -1: Retroceder, 1: Avanzar

Se definen las razones de probabilidad del dado cargado, lo que significa que las probailidades no son uniformes, algunas tienen más probabilidad de salir que otras. En lugar de un dado justo con una probabilidad uniforme de 1/6, aquí se asignan valores específicos: sacar un 1 o 6 tiene una probabilidad de 10%, un 2 o un 5 ocurre el 15% del tiempo, y un 3 o un 4 sucede con mayor frecuencia (20%). Esto afecta las transiciones del MDP, ya que los valores medios (3 y 4) serán más comunes, influyendo en la estrategia óptima del jugador al moverse en el tablero.

In [ ]:
probabilidades = [0.125, 0.175, 0.2, 0.2, 0.175, 0.125]  # Probabilidades del dado cargado acuerdo distribución normal

A continuación, se definen dos diccionarios en Python que modelan las escaleras y serpientes del tablero, donde las claves representan las casillas de inicio y los valores indican las casillas destino. El diccionario escaleras indica las casillas donde un jugador avanza automáticamente a una posición más alta, mientras que serpientes representa las casillas que obligan al jugador a retroceder, dificultando su progreso. Estas transiciones no lineales afectan la dinámica del juego y se incorporan en la función de probabilidad de transición del MDP, asegurando que el modelo refleje correctamente las reglas del juego.

In [6]:
# Definir posiciones de escaleras y serpientes
escaleras = {8: 26, 21: 82, 43: 77, 50: 91, 54: 93, 66: 87, 62: 96, 80: 100} # modelado de 8 escaleras
serpientes = {52: 11, 69: 33, 92: 51, 48: 9, 73: 1, 55: 7, 46: 5, 95: 24, 64: 36, 44: 22, 98: 28, 83: 19, 59: 17} # modelado de serpientes

Se definen mediante listas, los estados correspondientes a victoria o derrota denotados en la grafica de color azul y rojo respectivamente

In [7]:
# Casillas de victoria y derrota
estadoGanar = [80, 100]  # Casillas azules
estadoPerder = [23, 37, 45, 67, 89]  # Casillas rojas

Se modela la variable gamma que en el MDP permite dar relevancia a la evolución de los estados en el tiempo, el valor de 0.9 se parametriza para dar mayor relevancia a los estados recientes en comparación a los estados más antiguos

In [8]:
# Factor de descuento
gamma = 0.9

Se modela una función que modela las probabbilidades de transición en el talero de juego, esta calcula la distribución de probabilidad de transición en el tablero de escaleras y serpientes, modelando cómo un jugador se moverá desde un estado actual 𝑠 tras tomar una acción 𝑎 y lanzar el dado. Para cada posible resultado del dado, ajusta el estado futuro sumando o restando el valor obtenido según la acción tomada. Luego, maneja los rebotes en los extremos del tablero si el jugador supera la casilla 100 o cae por debajo de la casilla 1. Posteriormente, aplica las reglas de escaleras y serpientes, trasladando al jugador si cae en una de estas casillas. Finalmente, almacena la probabilidad acumulada de alcanzar cada estado en un vector, asegurando que la suma total de probabilidades sea 1, y lo retorna, permitiendo definir la matriz de transición 𝑃(𝑠′∣𝑠,𝑎) clave en el MDP.

In [9]:
def probTransicion(s, a, probabilidades):
    """ Calcula la distribución de probabilidad para el próximo estado."""
    prob_nuevoEstado = np.zeros(numEstados + 1)  # De 1 a 100
    
    for lanzamiento, prob in enumerate(probabilidades, start=1):
        nuevoEstado = s + a * lanzamiento
        
        # Rebote en los extremos
        if nuevoEstado > 100:
            nuevoEstado = 100 - (nuevoEstado - 100)
        elif nuevoEstado < 1:
            nuevoEstado = 1 + (1 - nuevoEstado)
        
        # Escaleras y serpientes
        if nuevoEstado in escaleras:
            nuevoEstado = escaleras[nuevoEstado]
        elif nuevoEstado in serpientes:
            nuevoEstado = serpientes[nuevoEstado]
        
        prob_nuevoEstado[nuevoEstado] += prob
    
    return prob_nuevoEstado

Se modela la función de recomensas, esta asigna una recompensa a cada estado 𝑠 según su impacto en el juego, retornando +1 si el jugador alcanza una casilla de victoria (estadoGanar), -1 si cae en una casilla de derrota (estadoPerder), y -0.01 en cualquier otro caso como penalización por movimiento, incentivando así estrategias que minimicen el número de pasos y maximicen la recompensa total en el MDP.

In [10]:
def funRecompensa(s):
    """ Define la función de recompensa."""
    if s in estadoGanar:
        return 1  # Victoria
    elif s in estadoPerder:
        return -1  # Derrota
    return -0.01  # Penalización por movimiento
